In [1]:
!pip install -U deepctr-torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 2.0 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/shenweichen/DeepCTR-Torch.git

Cloning into 'DeepCTR-Torch'...
remote: Enumerating objects: 1452, done.
remote: Counting objects: 100% (355/355), done.
remote: Compressing objects: 100% (230/230), done.
remote: Total 1452 (delta 220), reused 203 (delta 118), pack-reused 1097
Receiving objects: 100% (1452/1452), 5.45 MiB | 28.94 MiB/s, done.
Resolving deltas: 100% (929/929), done.


In [3]:
%cd DeepCTR-Torch


/content/DeepCTR-Torch


# Import Python Package

In [4]:
# -*- coding: utf-8 -*-
import pandas as pd
import torch
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *


# Import Data

In [5]:
data = pd.read_csv('/content/DeepCTR-Torch/examples/criteo_sample.txt')
data

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,NaN,3,260.0,NaN,17668.0,NaN,NaN,33.0,NaN,...,e5ba7672,87c6f83c,NaN,NaN,0429f84b,NaN,3a171ecb,c0d61a5c,NaN,NaN
1,0,NaN,-1,19.0,35.0,30251.0,247.0,1.0,35.0,160.0,...,d4bb7bd8,6fc84bfb,NaN,NaN,5155d8a3,NaN,be7c41b4,ded4aac9,NaN,NaN
2,0,0.0,0,2.0,12.0,2013.0,164.0,6.0,35.0,523.0,...,e5ba7672,675c9258,NaN,NaN,2e01979f,NaN,bcdee96c,6d5d1302,NaN,NaN
3,0,NaN,13,1.0,4.0,16836.0,200.0,5.0,4.0,29.0,...,e5ba7672,52e44668,NaN,NaN,e587c466,NaN,32c7478e,3b183c5c,NaN,NaN
4,0,0.0,0,104.0,27.0,1990.0,142.0,4.0,32.0,37.0,...,e5ba7672,25c88e42,21ddcdc9,b1252a9d,0e8585d2,NaN,32c7478e,0d4a6d1a,001f3601,92c878de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,NaN,0,113.0,3.0,3036.0,575.0,2.0,3.0,214.0,...,07c540c4,9880032b,21ddcdc9,5840adea,34cc61bb,c9d4222a,32c7478e,e5ed7da2,ea9a246c,984e0db0
196,1,0.0,1,1.0,1.0,1607.0,12.0,1.0,12.0,15.0,...,1e88c74f,3972b4ed,NaN,NaN,d1aa4512,NaN,32c7478e,9257f75f,NaN,NaN
197,1,1.0,0,6.0,3.0,0.0,0.0,19.0,3.0,3.0,...,3486227d,5aed7436,54591762,a458ea53,4a2c3526,NaN,32c7478e,1793a828,e8b83407,1a02cbe1
198,0,0.0,22,6.0,22.0,203.0,153.0,80.0,18.0,508.0,...,3486227d,13145934,55dd3565,5840adea,bf647035,NaN,32c7478e,1481ceb4,e8b83407,988b0775


# Data Preprocessing Pipline

In [6]:

sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]

data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['label']

## Discrete Data Processing

In [7]:
# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

In [8]:
data

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,0.0,3,260.0,0.0,17668.0,0.0,0.0,33.0,0.0,...,8,66,0,0,3,0,1,96,0,0
1,0,0.0,-1,19.0,35.0,30251.0,247.0,1.0,35.0,160.0,...,7,52,0,0,47,0,7,112,0,0
2,0,0.0,0,2.0,12.0,2013.0,164.0,6.0,35.0,523.0,...,8,49,0,0,25,0,6,53,0,0
3,0,0.0,13,1.0,4.0,16836.0,200.0,5.0,4.0,29.0,...,8,37,0,0,156,0,0,32,0,0
4,0,0.0,0,104.0,27.0,1990.0,142.0,4.0,32.0,37.0,...,8,14,5,3,9,0,0,5,1,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0.0,0,113.0,3.0,3036.0,575.0,2.0,3.0,214.0,...,0,74,5,1,30,5,0,118,17,48
196,1,0.0,1,1.0,1.0,1607.0,12.0,1.0,12.0,15.0,...,1,25,0,0,138,0,0,68,0,0
197,1,1.0,0,6.0,3.0,0.0,0.0,19.0,3.0,3.0,...,4,40,17,2,41,0,0,12,16,11
198,0,0.0,22,6.0,22.0,203.0,153.0,80.0,18.0,508.0,...,4,7,18,1,123,0,0,10,16,49


## Continuous Data Processing

In [9]:
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])

In [10]:
data

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,0.000000,0.001332,0.092362,0.000000,0.034825,0.000000,0.000000,0.673469,0.000000,...,8,66,0,0,3,0,1,96,0,0
1,0,0.000000,0.000000,0.006750,0.402299,0.059628,0.117284,0.003322,0.714286,0.154739,...,7,52,0,0,47,0,7,112,0,0
2,0,0.000000,0.000333,0.000710,0.137931,0.003968,0.077873,0.019934,0.714286,0.505803,...,8,49,0,0,25,0,6,53,0,0
3,0,0.000000,0.004664,0.000355,0.045977,0.033185,0.094967,0.016611,0.081633,0.028046,...,8,37,0,0,156,0,0,32,0,0
4,0,0.000000,0.000333,0.036945,0.310345,0.003922,0.067426,0.013289,0.653061,0.035783,...,8,14,5,3,9,0,0,5,1,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0.000000,0.000333,0.040142,0.034483,0.005984,0.273029,0.006645,0.061224,0.206963,...,0,74,5,1,30,5,0,118,17,48
196,1,0.000000,0.000666,0.000355,0.011494,0.003168,0.005698,0.003322,0.244898,0.014507,...,1,25,0,0,138,0,0,68,0,0
197,1,0.027027,0.000333,0.002131,0.034483,0.000000,0.000000,0.063123,0.061224,0.002901,...,4,40,17,2,41,0,0,12,16,11
198,0,0.000000,0.007662,0.002131,0.252874,0.000400,0.072650,0.265781,0.367347,0.491296,...,4,7,18,1,123,0,0,10,16,49


## Obtain the Feature of DNN or Linear 

In [11]:

# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].max() + 1, embedding_dim=4) for feat in sparse_features] + [DenseFeat(feat, 1, ) for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
feature_names

['C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21',
 'C22',
 'C23',
 'C24',
 'C25',
 'C26',
 'I1',
 'I2',
 'I3',
 'I4',
 'I5',
 'I6',
 'I7',
 'I8',
 'I9',
 'I10',
 'I11',
 'I12',
 'I13']

## Divide Train and Test Dataset

In [12]:

# 3.generate input data for model

train, test = train_test_split(data, test_size=0.2, random_state=2020)
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

# Check GPU CPU

In [13]:
device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# Construct DeepFM Framework 

In [14]:
# 4.Define Model,train,predict and evaluate

model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                task='binary',
                l2_reg_embedding=1e-5, device=device)

In [15]:
model.compile("adagrad", "binary_crossentropy",
              metrics=["binary_crossentropy", "auc"], )

history = model.fit(train_model_input, train[target].values, batch_size=32, epochs=10, verbose=2,
                    validation_split=0.2)
pred_ans = model.predict(test_model_input, 256)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))


cpu
Train on 128 samples, validate on 32 samples, 4 steps per epoch
Epoch 1/10
0s - loss:  0.6109 - binary_crossentropy:  0.6109 - auc:  0.5066 - val_binary_crossentropy:  0.5487 - val_auc:  0.4686
Epoch 2/10
0s - loss:  0.4932 - binary_crossentropy:  0.4932 - auc:  0.9366 - val_binary_crossentropy:  0.5458 - val_auc:  0.5029
Epoch 3/10
0s - loss:  0.3494 - binary_crossentropy:  0.3494 - auc:  0.9953 - val_binary_crossentropy:  0.5603 - val_auc:  0.5029
Epoch 4/10
0s - loss:  0.1821 - binary_crossentropy:  0.1821 - auc:  0.9987 - val_binary_crossentropy:  0.5931 - val_auc:  0.5029
Epoch 5/10
0s - loss:  0.0924 - binary_crossentropy:  0.0924 - auc:  1.0000 - val_binary_crossentropy:  0.6305 - val_auc:  0.5086
Epoch 6/10
0s - loss:  0.0560 - binary_crossentropy:  0.0560 - auc:  1.0000 - val_binary_crossentropy:  0.7164 - val_auc:  0.4800
Epoch 7/10
0s - loss:  0.0383 - binary_crossentropy:  0.0383 - auc:  1.0000 - val_binary_crossentropy:  0.7183 - val_auc:  0.4914
Epoch 8/10
0s - loss: 

# Construct DCN Framework

In [16]:
# 4.Define Model,train,predict and evaluate

model = DCN(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                task='binary',
                l2_reg_embedding=1e-5, device=device)

In [17]:
model.compile("adagrad", "binary_crossentropy",
              metrics=["binary_crossentropy", "auc"], )

history = model.fit(train_model_input, train[target].values, batch_size=32, epochs=10, verbose=2,
                    validation_split=0.2)
pred_ans = model.predict(test_model_input, 256)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))


cpu
Train on 128 samples, validate on 32 samples, 4 steps per epoch
Epoch 1/10
0s - loss:  0.6162 - binary_crossentropy:  0.6162 - auc:  0.5520 - val_binary_crossentropy:  0.5445 - val_auc:  0.3829
Epoch 2/10
0s - loss:  0.5010 - binary_crossentropy:  0.5010 - auc:  0.9932 - val_binary_crossentropy:  0.5425 - val_auc:  0.4457
Epoch 3/10
0s - loss:  0.4061 - binary_crossentropy:  0.4061 - auc:  0.9989 - val_binary_crossentropy:  0.5471 - val_auc:  0.4743
Epoch 4/10
0s - loss:  0.2773 - binary_crossentropy:  0.2773 - auc:  0.9987 - val_binary_crossentropy:  0.5641 - val_auc:  0.4686
Epoch 5/10
0s - loss:  0.1855 - binary_crossentropy:  0.1855 - auc:  1.0000 - val_binary_crossentropy:  0.6104 - val_auc:  0.4629
Epoch 6/10
0s - loss:  0.1293 - binary_crossentropy:  0.1293 - auc:  1.0000 - val_binary_crossentropy:  0.6283 - val_auc:  0.4514
Epoch 7/10
0s - loss:  0.0943 - binary_crossentropy:  0.0943 - auc:  1.0000 - val_binary_crossentropy:  0.6566 - val_auc:  0.4629
Epoch 8/10
0s - loss: 

# Construct xDeepFM

In [18]:
# 4.Define Model,train,predict and evaluate
model = xDeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                task='binary',
                l2_reg_embedding=1e-5, device=device)

In [19]:
model.compile("adagrad", "binary_crossentropy",
              metrics=["binary_crossentropy", "auc"], )

history = model.fit(train_model_input, train[target].values, batch_size=32, epochs=10, verbose=2,
                    validation_split=0.2)
pred_ans = model.predict(test_model_input, 256)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))


cpu
Train on 128 samples, validate on 32 samples, 4 steps per epoch
Epoch 1/10
0s - loss:  0.6199 - binary_crossentropy:  0.6199 - auc:  0.5597 - val_binary_crossentropy:  0.5355 - val_auc:  0.5600
Epoch 2/10
0s - loss:  0.5062 - binary_crossentropy:  0.5062 - auc:  0.9875 - val_binary_crossentropy:  0.5391 - val_auc:  0.5029
Epoch 3/10
0s - loss:  0.3582 - binary_crossentropy:  0.3582 - auc:  0.9964 - val_binary_crossentropy:  0.5511 - val_auc:  0.5086
Epoch 4/10
0s - loss:  0.1870 - binary_crossentropy:  0.1870 - auc:  1.0000 - val_binary_crossentropy:  0.6239 - val_auc:  0.4857
Epoch 5/10
0s - loss:  0.0874 - binary_crossentropy:  0.0874 - auc:  1.0000 - val_binary_crossentropy:  0.6788 - val_auc:  0.5029
Epoch 6/10
0s - loss:  0.0313 - binary_crossentropy:  0.0313 - auc:  1.0000 - val_binary_crossentropy:  0.7490 - val_auc:  0.4800
Epoch 7/10
0s - loss:  0.0115 - binary_crossentropy:  0.0115 - auc:  1.0000 - val_binary_crossentropy:  0.8159 - val_auc:  0.4857
Epoch 8/10
0s - loss: 

In [20]:
pred_ans

array([[0.27484444],
       [0.03867785],
       [0.20426035],
       [0.05335417],
       [0.17994112],
       [0.03212677],
       [0.16272531],
       [0.26865381],
       [0.2035709 ],
       [0.58597451],
       [0.05532624],
       [0.10799602],
       [0.11468782],
       [0.04047079],
       [0.00316433],
       [0.02826971],
       [0.00229461],
       [0.10530006],
       [0.05092963],
       [0.08276182],
       [0.68384826],
       [0.10195314],
       [0.02563304],
       [0.29763857],
       [0.09287351],
       [0.53201616],
       [0.00215066],
       [0.00112963],
       [0.07616416],
       [0.1202509 ],
       [0.19295989],
       [0.10268123],
       [0.00300922],
       [0.10079313],
       [0.10221554],
       [0.10892526],
       [0.31048042],
       [0.07398373],
       [0.00370819],
       [0.16753459]])

In [21]:
pd.DataFrame(pred_ans).to_csv('result.csv')
